In [ ]:
!pip install wandb
!wandb login 365a84ddf123ea8d9236af9bcf504ded270575e7

import wandb
wandb.sagemaker_auth(path='.')

In [ ]:
import json
import datetime
import sagemaker
from sagemaker.pytorch import PyTorch
import os

experiment_name = 'experiment-meaningful-description'

In [ ]:
config = json.load(open('config.json'))
config["lr"] = 3e-4
config["batch_size"] = 64
config["gamma"] = 0.99
config["steps"] = 0.1e6
config["activation_fn"] = "relu"
config["net_arch"] = [256,256]
json.dump(config, open('config.json',"w"), indent=4)

In [ ]:
estimator = PyTorch(
    entry_point='train.py',
    source_dir='.',
    role=sagemaker.get_execution_role(),
    framework_version='1.5.0',
    py_version="py3",  # now required
    instance_type="ml.c4.xlarge",
    instance_count=1,
    # volume_size=30,
    use_spot_instances=True,
    max_run=86000,
    max_wait=86400,
    debugger_hook_config=False,
    hyperparameters={'backend': 'gloo'},
)

In [ ]:
estimator.fit(
    job_name=experiment_name + datetime.datetime.now().strftime("-%Y-%m-%d-%H-%M-%S"),
)